In [1]:
!ls runs |wc -l

     133


In [1]:
import os
import pandas as pd
from tensorflow.python.summary.summary_iterator import summary_iterator
from tqdm import tqdm
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import clipboard

# Read data

In [2]:
def parse_tfevent(tfevent, filepath):
    return dict(
        fname = filepath,
        wall_time=tfevent.wall_time,
        name=tfevent.summary.value[0].tag,
        step=tfevent.step,
        value=tfevent.summary.value[0],
    )


In [3]:
all_files = []
main_path =Path("runs")
for (root, _, filenames) in os.walk(main_path):
    for filename in filenames:
        if "events.out.tfevents" in filename:
            all_files.append(root/Path(filename))
print(len(all_files))

183


In [4]:
full_dict = {}

In [18]:
for files in tqdm(all_files, total = len(all_files)):
    temp_dict = {}
    for ev in summary_iterator(str(files)):
        try:
            parsed = parse_tfevent(ev, files)
            # print(parsed)
            value = parsed["value"]
            # print(value.tensor.string_val[0].decode('ascii'))
            # print("here0")

            # value = value.tag
            try:
                # value = value.tag

                value_str = value.tensor.string_val[0].decode('ascii')
            except:
                try:
                    value_str = value.simple_value[-1]
                except:

                    value_str = value.simple_value
            # print(value)
            # print("here1")
            # try:
            #     temp_dict[value.tag].append(value_str)
            # except Exception as e:
            #     # print(e)
            temp_dict[value.tag.replace("/text_summary", "")] = value_str
        except Exception as e:
            # print(e)
            pass
    full_dict[str(files)] = temp_dict

100%|██████████| 183/183 [00:00<00:00, 608.53it/s]


In [19]:
combined_df = pd.DataFrame.from_records(full_dict).T.reset_index()

In [20]:
combined_df.head()

,index,experiment_name,ds_name,image_size,batch_size,enable_proxy_attention,change_subset_attention,num_gpu,num_cpu,transfer_imagenet,...,num_classes,proxy_step,Loss/Train,Acc/Train,Loss/Val,Acc/Val,save_path,Number_Chosen,original_images,converted_proxy
0,"runs/asl_blend_test+08032023_16:30:00_ps-[10, ...",blend_test,asl,224,32,True,0.3,1,10,True,...,29,0.0,0.478703,0.85625,0.115334,0.96525,NaN,NaN,NaN,NaN
1,"runs/asl_blend_test+08032023_16:30:00_ps-[10, ...",blend_test,asl,224,32,True,0.3,1,10,True,...,29,1.0,0.510605,0.84025,0.14249,0.95675,/mnt/e/CODE/Github/improving_robotics_datasets...,10.0,0.0,0.0
2,"runs/asl_blend_test+08032023_16:30:00_ps-[10, ...",blend_test,asl,224,32,True,0.3,1,10,True,...,29,0.0,0.367487,0.88625,0.036445,0.98925,/mnt/e/CODE/Github/improving_robotics_datasets...,NaN,NaN,NaN
3,"runs/asl_blend_test+08032023_16:38:29_ps-[10, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"runs/asl_blend_test+08032023_16:38:51_ps-[10, ...",blend_test,asl,224,32,True,0.8,1,10,True,...,29,0.0,0.493784,0.85425,0.111464,0.96925,NaN,NaN,NaN,NaN


In [8]:
combined_df.shape

(183, 36)

In [37]:
combined_df["global_run_count"].value_counts()

10    27
11    27
Name: global_run_count, dtype: int64

In [28]:
combined_df[combined_df["global_run_count"]!= "0"]["proxy_steps"].value_counts()

[10, 'p', 10]    54
Name: proxy_steps, dtype: int64

# Preprocess

In [29]:
def check_proxy(string): return "p" in string

In [30]:
def calc_stats(values):
    return f"min: {values.min()} \nmax: {values.max()} \navg: {values.mean()}"

In [31]:
def convert_float(df, cols, totype= float):
    for col in cols:
        df[col] = df[col].astype(totype)

In [32]:
combined_df = combined_df.fillna(0)
# col to check for proxy
# combined_df["has_proxy"] = combined_df["proxy_steps"].apply(check_proxy)
# Fix naming
combined_df = combined_df.rename(columns={"Acc/Val":"accuracy", "proxy_steps":"step_schedule"})
# Fix types
convert_float(combined_df, ["change_subset_attention", "proxy_threshold", "accuracy"], float)
convert_float(combined_df, ["global_run_count"], int)
convert_float(combined_df, ["transfer_imagenet"], bool)

# ignore failed runs
combined_df = combined_df[combined_df["global_run_count"]!=0]


# Grouped Results

In [33]:
def return_grouped_results(df, group_cols ,filter = None, index_cols = (["ds_name", ("accuracy")]), print_latex = False):
    if filter != None:
        df = df.reset_index()
        for key in filter.keys():
            df = df[df[key] == filter[key]]
    final_df = pd.DataFrame(df.groupby(group_cols, as_index=True).mean(numeric_only = True)["accuracy"]).sort_values(index_cols, ascending=False)
    if print_latex == True:
        clipboard.copy(final_df.to_latex())

    return final_df



In [34]:
return_grouped_results(combined_df, ["ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "step_schedule"], filter = {"transfer_imagenet":True , "ds_name":"asl"})

,,,,,accuracy
ds_name,pixel_replacement_method,transfer_imagenet,gradient_method,step_schedule,
asl,blended,True,gradcamplusplus,"[10, 'p', 10]",0.966371


In [35]:
return_grouped_results(combined_df, ["ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "step_schedule", "global_run_count"])

accuracy
ds_name    pixel_replacement_method transfer_imagenet gradient_method step_schedule global_run_count          
imagenette blended                  True              gradcamplusplus [10, 'p', 10] 11                0.710021
                                                                                    10                0.662625
asl        blended                  True              gradcamplusplus [10, 'p', 10] 11                0.969076
                                                                                    10                0.963667

In [36]:
return_grouped_results(combined_df, ["ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "change_subset_attention", "step_schedule"])

accuracy
ds_name    pixel_replacement_method transfer_imagenet gradient_method change_subset_attention step_schedule          
imagenette blended                  True              gradcamplusplus 0.8                     [10, 'p', 10]  0.686323
asl        blended                  True              gradcamplusplus 0.3                     [10, 'p', 10]  0.973000
                                                                      0.8                     [10, 'p', 10]  0.963057

## Parameter elimination

In [428]:
def check_wrt_accuracy(has_proxy, condition, head = None):
    grouped_vals = has_proxy.groupby([condition, "ds_name", "transfer_imagenet"]).mean(numeric_only = True)["accuracy"].reset_index().sort_values(by = ["accuracy"], ascending=False)
    print("Transfer")
    if head != None:
        print(grouped_vals[grouped_vals["transfer_imagenet"] == True].head(head).to_markdown())
        print("No Transfer")
        print(grouped_vals[grouped_vals["transfer_imagenet"] == False].head(head).to_markdown())
    else:
        print(grouped_vals[grouped_vals["transfer_imagenet"] == True].to_markdown())
        print("No Transfer")
        print(grouped_vals[grouped_vals["transfer_imagenet"] == False].to_markdown())

In [429]:
check_wrt_accuracy(combined_df, "clear_every_step")

Transfer
|    | clear_every_step   | ds_name    | transfer_imagenet   |   accuracy |
|---:|:-------------------|:-----------|:--------------------|-----------:|
|  0 | False              | asl        | True                |   0.965529 |
|  2 | True               | asl        | True                |   0.926681 |
|  1 | False              | imagenette | True                |   0.816    |
|  3 | True               | imagenette | True                |   0.66642  |
No Transfer
| clear_every_step   | ds_name   | transfer_imagenet   | accuracy   |
|--------------------|-----------|---------------------|------------|


In [430]:
check_wrt_accuracy(combined_df, "change_subset_attention")

Transfer
|    |   change_subset_attention | ds_name    | transfer_imagenet   |   accuracy |
|---:|--------------------------:|:-----------|:--------------------|-----------:|
|  4 |                       0.8 | asl        | True                |   0.957823 |
|  0 |                       0.3 | asl        | True                |   0.935525 |
|  2 |                       0.5 | asl        | True                |   0.911833 |
|  1 |                       0.3 | imagenette | True                |   0.771125 |
|  3 |                       0.5 | imagenette | True                |   0.769875 |
|  5 |                       0.8 | imagenette | True                |   0.663041 |
No Transfer
| change_subset_attention   | ds_name   | transfer_imagenet   | accuracy   |
|---------------------------|-----------|---------------------|------------|


In [432]:
check_wrt_accuracy(combined_df, "gradient_method")

Transfer
|    | gradient_method   | ds_name    | transfer_imagenet   |   accuracy |
|---:|:------------------|:-----------|:--------------------|-----------:|
|  0 | gradcamplusplus   | asl        | True                |   0.938634 |
|  1 | gradcamplusplus   | imagenette | True                |   0.674157 |
No Transfer
| gradient_method   | ds_name   | transfer_imagenet   | accuracy   |
|-------------------|-----------|---------------------|------------|


In [433]:
check_wrt_accuracy(combined_df, "pixel_replacement_method")

Transfer
|    | pixel_replacement_method   | ds_name    | transfer_imagenet   |   accuracy |
|---:|:---------------------------|:-----------|:--------------------|-----------:|
|  0 | blended                    | asl        | True                |   0.938634 |
|  1 | blended                    | imagenette | True                |   0.674157 |
No Transfer
| pixel_replacement_method   | ds_name   | transfer_imagenet   | accuracy   |
|----------------------------|-----------|---------------------|------------|


In [434]:
check_wrt_accuracy(combined_df, "proxy_threshold", head= 4)

Transfer
|    |   proxy_threshold | ds_name   | transfer_imagenet   |   accuracy |
|---:|------------------:|:----------|:--------------------|-----------:|
| 77 |          0.92068  | asl       | True                |    0.98875 |
| 84 |          0.94807  | asl       | True                |    0.9825  |
| 71 |          0.898883 | asl       | True                |    0.98    |
| 75 |          0.913456 | asl       | True                |    0.975   |
No Transfer
| proxy_threshold   | ds_name   | transfer_imagenet   | accuracy   |
|-------------------|-----------|---------------------|------------|


In [435]:

check_wrt_accuracy(combined_df, "subset_images")

Transfer
|    |   subset_images | ds_name    | transfer_imagenet   |   accuracy |
|---:|----------------:|:-----------|:--------------------|-----------:|
|  0 |            8000 | asl        | True                |   0.938634 |
|  1 |            8000 | imagenette | True                |   0.674157 |
No Transfer
| subset_images   | ds_name   | transfer_imagenet   | accuracy   |
|-----------------|-----------|---------------------|------------|


In [437]:

check_wrt_accuracy(combined_df, "step_schedule")

Transfer
|    | step_schedule   | ds_name    | transfer_imagenet   |   accuracy |
|---:|:----------------|:-----------|:--------------------|-----------:|
|  4 | [21]            | asl        | True                |   0.976833 |
|  0 | [10, 'p', 10]   | asl        | True                |   0.962566 |
|  2 | [20, 'p', 10]   | asl        | True                |   0.88725  |
|  6 | [31]            | asl        | True                |   0.87725  |
|  5 | [21]            | imagenette | True                |   0.809167 |
|  1 | [10, 'p', 10]   | imagenette | True                |   0.669798 |
|  7 | [31]            | imagenette | True                |   0.62725  |
|  3 | [20, 'p', 10]   | imagenette | True                |   0.602167 |
No Transfer
| step_schedule   | ds_name   | transfer_imagenet   | accuracy   |
|-----------------|-----------|---------------------|------------|
